In [1]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [3]:
### IMPORTS ###
import numpy as np
import os
import json

In [4]:
### CLASSES ###
class Cell:
    def __init__(self, input_val):
        self.value = 0
        if(input_val != '*'):
            self.value = int(input_val)


class Board:
    def __init__(self, file_name):
        # Get Boards folder
        current_directory = os.getcwd() #Current working directory
        parent_directory = os.path.dirname(current_directory)
        new_folder_path = os.path.join(parent_directory, 'Unsolved-Boards')
        text_file_path = os.path.join(new_folder_path, file_name)

        # Check if the folder exists, if not, create it
        if not os.path.exists(text_file_path):
            print("ERROR - the path " + text_file_path + " does not exist")
            return None

        
        # Read File
        fileReader = open(text_file_path, "r")    
        file_as_text = fileReader.read()
        text_to_array = file_as_text.split('\n')
        text_to_array = np.array([[tile for tile in row] for row in text_to_array if len(row) > 0])
        #print(text_to_array)
        
        
        # Turn File into Board object
        self.board = []
        
        for row in text_to_array:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
            
    def set_json(self, json_data, val):
        self.board = []
        
        for row in text_to_array:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
            
    def updateToSolved(self, gameBoard):
        self.board = []
        
        for row in gameBoard:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
        
    def print_board(self):
        for i, row in enumerate(self.board): 
            
            if i % 3 == 0 and i != 0:
                print("-" * 7 + "+" + "-" * 7 + "+" + "-" * 7)  # Print horizontal line after every 3 rows

            print(" ", end="")
            for j, cell in enumerate(row):
                if j % 3 == 0 and j != 0:
                    print("| ", end="")  # Print vertical line after every 3 cells

                if cell.value == 0:
                    print("*", end=" ")
                else:
                    print(cell.value, end=" ")

            print()  # Move to the next line after printing a row
            
    def toJSON(self):
        json_data = {'board': []}
        for row in self.board:
            row_data = []
            for cell in row:
                row_data.append(cell.value)
            json_data['board'].append(row_data)
        return json.dumps(json_data)
    
    def copy(self):
        newBoardList = []
        for row in self.board:

            row_list = [] 
            for cell in row:
                row_list.append(cell.value)

            newBoardList.append(row_list)

        return newBoardList  

In [5]:
### BACK TRACKING ALGORTHM ###

#Creating the Constraint Satisfaction Problem variable:
sudoku_csp = {
    #The variable item contains a list of the cells of the Sudoku Board
    'variables': [(i, j) for i in range(9) for j in range(9)],
    
    #The domain item contains a dictionary of each variable containing a list of numbers from 1 to 9
    'domains': {variable: list(range(1, 10)) for variable in [(i, j) for i in range(9) for j in range(9)]},
    
    #The constraint item contains a dictionary of each constraint type
    #Each constraint holds a lambda function for checking if a given value violates the constraint or not
    'constraints': {
        'row': lambda board, row, col, num: num not in board[row],
        'column': lambda board, row, col, num: num not in [board[i][col] for i in range(9)],
        'subgrid': lambda board, row, col, num: all(num != board[i][j] for i in range(3*(row//3), 3*(row//3) + 3) for j in range(3*(col//3), 3*(col//3) + 3))
    }
}

In [6]:
#The class for solving the Puzzle
def solve_sudoku_csp(board, csp):
    
    #Creating variables holding the different components of the CSP variable
    variables = csp['variables']
    domains = csp['domains']
    constraints = csp['constraints']
    
    #Creating a dictionary to store the assigned cells
    assigned_cells = {}  
    
    #the backtracking function
    def backtrack(board):
        
        #Calls the function to find an unassigned cell
        empty_cell = find_empty_cell(board)
        
        #if the empty cell returns None, the board is solved
        if not empty_cell:
            return True
        
        #set the row and column of the empty cell
        row, col = empty_cell
        
        #go through each value in the domain of the empty cell to find the correct value
        for num in domains[empty_cell]:
            
            #if the current value does not violate any constraint, assign the value to the empty cell
            if is_valid(board, row, col, num, constraints):
                board[row][col] = num
                
                #Record the assigned cell
                assigned_cells[(row, col)] = num
                
                #call the backtracking function for another empty cell
                #if there are no more empty cells, return true
                if backtrack(board):
                    return True
                #Backtrack and remove the cell
                board[row][col] = 0  
                del assigned_cells[(row, col)] 
                
        #No solution found
        return False  
    
    #the function to find an empty cell
    def find_empty_cell(board):
        
        #Go through each row and column to find a cell that contains 0
        for i in range(9):
            for j in range(9):
                if board[i][j] == 0:
                    return (i, j)
        return None
    
    #the function to check the board's consistency with the constraints
    def is_valid(board, row, col, num, constraints):
        
        #looping to go through each constraint and using the function inside to test for consistency
        for constraint_type, constraint_func in constraints.items():
            
            #if the function does not return true, then it is not consistent
            if not constraint_func(board, row, col, num):
                return False
        return True
    
    #if the backtracking function returns true, return the board
    if backtrack(board):
        return True, board, assigned_cells  # Return assigned cells along with the solution
    else:
        return False, None, None



In [7]:
### SOLVING STRATEGIES ###



In [8]:
### COMBINED SOLVING AND BACKTRACKING STRATEGIES ###
def solveBoard(boardName):
    print("----------- " + boardName + " -----------")
    
    #Get Copy of Unsolved Board
    gameBoard = Board(boardName)
    unsolvedBoard = gameBoard.copy()
    
    print("\nGame Board\n")
    gameBoard.print_board()
    print('\n')

    #Solve
    solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)   
    gameBoard.updateToSolved(solution)
    
    #Shows the board if it is solved
    if solved:
        print("Sudoku Solved Successfully!\n")
        gameBoard.print_board()
            
        return solved, gameBoard, assignments
    else:
        print("No solution exists!")
        return solved, None, None
        

### Testing ###
gameBoards = ["Puzzle_1.txt", "Puzzle_2.txt", "Puzzle_3.txt", "Puzzle_4.txt", "Puzzle_5.txt"]
for game in gameBoards:
    solveBoard(game)
    print('\n\n\n')

----------- Puzzle_1.txt -----------

Game Board

 8 2 1 | * * * | * * 7 
 * * * | 8 * * | * 6 * 
 * 6 * | 9 3 * | * * 5 
-------+-------+-------
 * * 8 | 2 * 1 | 6 * * 
 * * * | 7 * * | 2 8 4 
 2 4 * | 6 * 3 | 7 * * 
-------+-------+-------
 6 * 5 | * * * | 1 * 3 
 * 7 * | * 5 * | * * * 
 9 1 2 | * * * | * * 6 


Sudoku Solved Successfully!

 8 2 1 | 5 6 4 | 3 9 7 
 5 9 3 | 8 1 7 | 4 6 2 
 4 6 7 | 9 3 2 | 8 1 5 
-------+-------+-------
 7 5 8 | 2 4 1 | 6 3 9 
 1 3 6 | 7 9 5 | 2 8 4 
 2 4 9 | 6 8 3 | 7 5 1 
-------+-------+-------
 6 8 5 | 4 2 9 | 1 7 3 
 3 7 4 | 1 5 6 | 9 2 8 
 9 1 2 | 3 7 8 | 5 4 6 




----------- Puzzle_2.txt -----------

Game Board

 5 3 * | * 7 * | * * * 
 6 * * | 1 9 5 | * * * 
 * 9 8 | * * * | * 6 * 
-------+-------+-------
 8 * * | * 6 * | * * 3 
 4 * * | 8 * 3 | * * 1 
 7 * * | * 2 * | * * 6 
-------+-------+-------
 * 6 * | * * * | 2 8 * 
 * * * | 4 1 9 | * * 5 
 * * * | * 8 * | * 7 9 


Sudoku Solved Successfully!

 5 3 4 | 6 7 8 | 9 1 2 
 6 7 2 | 1 9 5 | 3

In [9]:
### FRONT END CONNECTION ###
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import pandas as pd

In [36]:
### FRONT END TESTING ###
def create_json2():
    updates = [
        {
            "solve_type": "Solved cells",
            "eliminations": [{"x": 4, "y": 5, "values": [5]}],
            "found_value": 5,
            "group": None
        },
        {
            "solve_type": "Naked Single",
            "group": [{"x": 3, "y": 4, "values": [1, 2, 3, 5, 8]}],
            "eliminations": [{"x": 5, "y": 4, "values": [1, 9]}],
            "found_value": None
        },
        {
            "solve_type": "Hidden Single",
            "group": [{"x": 1, "y": 7, "values": [2, 3, 5, 6, 9]}],
            "eliminations": [{"x": 1, "y": 7, "values": [2, 6, 9]}],
            "found_value": {"x": 1, "y": 7, "value": 8}
        },
        {
            "solve_type": "Naked Pair",
            "group": [
                {"x": 2, "y": 6, "values": [2, 5]},
                {"x": 2, "y": 8, "values": [2, 5]}
            ],
            "eliminations": [
                {"x": 2, "y": 3, "values": [2, 5]},
                {"x": 2, "y": 5, "values": [2, 5]}
            ],
            "found_value": None
        }
    ]
    
    json_data = {"updates": updates}
    return json.dumps(json_data, indent=2)







def create_json(gameBoard, addedVals):
    added_json = addedValsToJson(addedVals)
    finalBoard_json = combine_json(gameBoard.toJSON(), added_json, 3)
    
    send_json = combine_json(create_json2(), finalBoard_json, 4)
    print(send_json)
    
    return send_json

def addedValsToJson(addedVals):
    added = [
        
    ]
    
    for coord, val in addedVals.items():
        added_val = {"val": val, "coord": coord}
        added.append(added_val)
    
    json_data = {"added": added}
    return json.dumps(json_data, indent=2)


def combine_json(json_1, json_2, indentSize):
    # Parse JSON strings into dictionaries
    data_1 = json.loads(json_1)
    data_2 = json.loads(json_2)
    
    # Combine the data
    combined_data = {**data_1, **data_2}
    
    # Convert combined data back to JSON string
    combined_json = json.dumps(combined_data, indent=indentSize)
    
    return combined_json



In [ ]:
app = Flask(__name__)
CORS(app)

# Define the route to serve HTML files from a folder
@app.route('/Front-End/<path:path>')
def serve_html(path):
    print("--- Html Page ---")
    current_directory = os.getcwd()  # Current working directory
    parent_directory = os.path.dirname(current_directory)
    frontEnd_dir = os.path.join(parent_directory, 'Front-End')

    print(frontEnd_dir + '/' + path)
    return send_from_directory(frontEnd_dir, path)

@app.route('/load', methods=['POST'])
def load():
    print('--- Loading ---')
    
    # Retrieve the file name from the request data or URL parameters
    requestData = request.get_json()
    fileName = requestData.get('fileName') # Assuming the file name is sent in a form field
    print(fileName)

    # Process the request to load data from the file (e.g., using Board(fileName))
    temp = Board(fileName)
    if(temp == None):
        return jsonify({'error': f'{fileName} Not Found'}), 404
    
    return temp.toJSON()


@app.route('/customBoard', methods=['POST'])
def customBoard():
    print("--- Custom Board ---")
    # Get the JSON data from the request
    #request_data = request.get_json()

    # Extract the file name and sudoku board data from the JSON data
    #board = Board("none")
    #board.set_json(request_data.get('sudokuBoard'))

    # Process the POST request and return JSON response
    result = create_json2()
    return jsonify(result)


@app.route('/solve', methods=['POST'])
def solve():
    print('--- Solving ---')
    
    solved, gameBoard, addedVals = solveBoard("Puzzle_1.txt")
    return jsonify(create_json(gameBoard, addedVals))
    
    #print(create_json(gameBoard, addedVals))
    
    if(solved):
        print('Solved')
        result = create_json2()
        return jsonify(create_json(gameBoard, addedVals))
    
    print('Not Solved')
    result = create_json2()
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True, port=8080, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2024 17:35:03] "GET /Front-End/index.html HTTP/1.1" 304 -
127.0.0.1 - - [17/Apr/2024 17:35:03] "POST /solve HTTP/1.1" 200 -


--- Html Page ---
C:\Users\rhigginso\Desktop\Class Folders\3) Junior - Spring\Artificial Intelligence\LTU-AI-Group_Project\Front-End/index.html
--- Solving ---
----------- Puzzle_1.txt -----------

Game Board

 8 2 1 | * * * | * * 7 
 * * * | 8 * * | * 6 * 
 * 6 * | 9 3 * | * * 5 
-------+-------+-------
 * * 8 | 2 * 1 | 6 * * 
 * * * | 7 * * | 2 8 4 
 2 4 * | 6 * 3 | 7 * * 
-------+-------+-------
 6 * 5 | * * * | 1 * 3 
 * 7 * | * 5 * | * * * 
 9 1 2 | * * * | * * 6 


Sudoku Solved Successfully!

 8 2 1 | 5 6 4 | 3 9 7 
 5 9 3 | 8 1 7 | 4 6 2 
 4 6 7 | 9 3 2 | 8 1 5 
-------+-------+-------
 7 5 8 | 2 4 1 | 6 3 9 
 1 3 6 | 7 9 5 | 2 8 4 
 2 4 9 | 6 8 3 | 7 5 1 
-------+-------+-------
 6 8 5 | 4 2 9 | 1 7 3 
 3 7 4 | 1 5 6 | 9 2 8 
 9 1 2 | 3 7 8 | 5 4 6 
{
    "updates": [
        {
            "solve_type": "Solved cells",
            "eliminations": [
                {
                    "x": 4,
                    "y": 5,
                    "values": [
                        